<a href="https://colab.research.google.com/github/maarongross/apriltag-imgs/blob/master/AprilTags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pip._internal import main as pipmain
pipmain(['install', 'apriltag'])
pipmain(['install', 'dxfgrabber'])

In [891]:
import cv2
from google.colab.patches import cv2_imshow
import apriltag
import dxfgrabber
import numpy as np
import time

In [ ]:
img = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/Track+Tags.PNG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(4208,3120), interpolation = cv2.INTER_AREA)
apriltag.DetectorOptions()
options = apriltag.DetectorOptions(families="tag16h5",quad_decimate=1)
detector = apriltag.Detector(options)
result = detector.detect(img)

img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

for r in result:

  (A,B,C,D) = r.corners 
  corner_B = (int(B[0]), int(B[1]))
  corner_C = (int(C[0]), int(C[1]))
  corner_D = (int(D[0]), int(D[1]))
  corner_A = (int(A[0]), int(A[1]))
  
  cv2.line(img, corner_A, corner_B, (0,255,0), 5)
  cv2.line(img, corner_B, corner_C, (0,255,0), 5)
  cv2.line(img, corner_C, corner_D, (0,255,0), 5)
  cv2.line(img, corner_D, corner_A, (0,255,0), 5)

  (cX,cY) = (int(r.center[0]), int(r.center[1]))
  cv2.circle(img,(cX,cY), 5, (0, 0, 255), -1)

  tagFamily = r.tag_family.decode("utf-8")
  tagID = str(r.tag_id)
  print(tagID)
  cv2.putText(img, "tagID: " + tagID, (corner_A[0]+50, corner_A[1] - 30),
              cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 10)
  
  print("[INFO] tag ID: {}".format(tagID))

cv2_imshow(img)

In [115]:
try:
  checkPointInfo = {10 : "Velocity = 10 \n Checkpoint #1",
                    20 : "Velocity = 50 \n Checkpoint #2",
                    30 : "Velocity = 10 \n Checkpoint #3"}
  print(checkPointInfo[int(tagID)])
except:
  print('No mapped check-point detected')



No mapped check-point detected


Want to track car location on a physical map

In [967]:
# Motion Parameters
freq = 20         #Hz
vel_const = 0.1   #m/s

# Loading track map and resizing
track = cv2.imread('/content/drive/MyDrive/(1) UCSD/(6) Fall 2022/MAE-148/AprilTags/track backround.png')
track = cv2.resize(track,(1920,1080), interpolation = cv2.INTER_AREA)

# Image Zero-Zero
(zX, zY) = (int(1920/2), int(1080/2))
pic_center = (zX, zY)

class polyLine():
  def __init__(self):
    self.header = None

class obj():
  objType = None
  pic_center = (-1,-1) #center of the picture where this obj is located  
  start = (-1,-1) # coordinates in meters from center of the track
  end = (-1,-1) # coordinates in meters from the center of the track
  radius_point = (-1,-1) # if the obj is a circle, location of the radius point
  angle = -1 # angle in radians measured from the horizontal
  distance = -1 # linear distance between begining and end point
  radius = -1 # if the object is a circle, it will need a radius value
  arc_orientation = -1
  color = (-1,-1,-1) # color to display the object as (b, g, r)
  
  theta = -1 

  linearPos = 0 # Linear position allong the object
  linearExtent = 0 # Total linear distance of the object

  prevObj = None # To carry over motion to the next object
  nextObj = None # To carry over motion to the previous object

  def __init__(self, object_type, pic_center, start, end, color, radius_point, arc_orientation, approach_direction):
    self.objType = object_type
    self.pic_center = (pic_center[0], pic_center[1])
    self.start = (start[0],start[1])
    self.realPos = (start[0], start[1])
    self.end = (end[0], end[1])
    self.color = color
    self.radius_point = (radius_point[0], radius_point[1])
    self.arc_orientation = arc_orientation
    self.approach_direction = approach_direction

    if self.objType == 'line':
      self.getAngle()
      self.getDistance()
    if self.objType == 'circle':
      self.getRadius()
      self.get_start_angle()
      self.get_arc_length()

  def getAngle(self):
    a = (self.end[0]-self.start[0], self.end[1]-self.start[1])
    anorm = (a[0]/np.sqrt(abs(np.square(a[0])+np.square(a[1]))), 
             a[1]/np.sqrt(abs(np.square(a[0])+np.square(a[1]))))
    self.angle = np.arccos(anorm[0])

    if anorm[1] < 0:
      self.angle = -self.angle
    else:
      pass
  def getDistance(self): #gets distance in meters
    a = (self.end[0]-self.start[0],self.end[1]-self.start[1])
    self.linearExtent = np.sqrt(np.square(a[0])+np.square(a[1]))
  def getRadius(self): #gets radius value in pixles
    xDiff = np.square(self.radius_point[0]-self.start[0])
    yDiff = np.square(self.radius_point[1]-self.start[1])
    self.radius = int(np.sqrt(xDiff + yDiff))
  def trans_obj_2_img(self,x,y):
    return((int(self.pic_center[0]+x), int(self.pic_center[1]+y)))
  def trans_img_2_obj(self,x,y):
    return((int(x-self.pic_center[0]), int(y-self.pic_center[1])))
  def trans_scaler_obj_2_img(self,x):
    return((int(self.pic_center[0]+x)))
  def trans_scaler_img_2_obj(self,x):
    return((int(x-self.pic_center[0])))
  def draw_start_end(self):
    #get start and end points
    (sX,sY) = self.trans_obj_2_img(self.start[0],self.start[1])
    (eX,eY) = self.trans_obj_2_img(self.end[0],self.end[1])

    #draw start and end points
    cv2.circle(track,(sX,sY), 5, self.color, -1)
    cv2.circle(track,(eX,eY), 5, self.color, -1)  
  def draw_radius_point(self):
    if self.objType == 'circle':
      (cX, cY) = self.trans_obj_2_img(self.radius_point[0],self.radius_point[1])
      cv2.circle(track, (cX,cY), 5, self.color, -1)
    else:
      pass
  def draw_path(self):
    if self.objType == 'line':
      cv2.line(track, self.trans_obj_2_img(self.start[0], self.start[1]), self.trans_obj_2_img(self.end[0], self.end[1]), self.color, 2)
    if self.objType == 'circle':
      (cX, cY) = self.trans_obj_2_img(self.radius_point[0],self.radius_point[1])
      cv2.circle(track, (cX,cY), self.radius, self.color, 2)
    else:
      pass
  def get_start_angle(self):    
      if self.objType == 'circle':
        (local_x, local_y) = (self.start[0]-self.radius_point[0], self.start[1]-self.radius_point[1])
        local_theta = np.arcsin(local_y/self.radius)
        if local_x < 0:
          local_theta = np.pi + local_theta 

        self.theta = local_theta
      else:
        pass
  def get_arc_length(self):
    (aX,aY) = (self.start[0] - self.radius_point[0] , self.start[1] - self.radius_point[1])
    (bX,bY) = (self.end[0] - self.radius_point[0], self.end[1] - self.radius_point[1])
    aMag = np.sqrt(abs(np.square(aX)+np.square(aY)))
    bMag = np.sqrt(abs(np.square(bX)+np.square(bY)))
    (aX_norm, aY_norm) = (aX/aMag, aY/aMag) 
    (bX_norm, bY_norm) = (bX/bMag, bY/bMag)
    phi = np.arccos(aX_norm*bX_norm + aY_norm*bY_norm)
    
    if self.arc_orientation == 1:
      phi = 2*np.pi - phi
    else:
      pass
    
    self.linearExtent = self.radius*phi

  def getMotion(self, velocity, time):
    if self.objType == 'line':

      #check if valid motion
      delta =  float(self.linearPos) + trans_dist2img(velocity*time)
      if delta < self.linearExtent and delta > 0:
        self.linearPos += trans_dist2img(velocity*time)
      if delta >= self.linearExtent:
        if self.nextObj is not None:
          runoff = delta - self.linearExtent
          self.nextObj.getMotion(trans_img2dist(runoff), 1)
          self.linearPos = 0
          self.realPos = self.start
          print('traversing forward')
          return trans_img2dist(runoff)
          pass
        else:
          pass
      if (self.linearPos + trans_dist2img(velocity*time) < 0):
        print('traversing backwards is not yet supported ¯\_(ツ)_/¯')
        pass

      #Start Motion  
      (rX, rY) = self.realPos
      
      self.realPos = (int(rX + trans_dist2img(velocity*time*np.cos(self.angle))), 
                      int(rY + trans_dist2img(velocity*time*np.sin(self.angle))))
      
      (cX,cY) = self.realPos
      #self.realPos = self.trans_obj_2_img(cX,cY)
      
      cv2.circle(track, self.trans_obj_2_img(cX,cY), 5, self.color, -1)

    if self.objType == 'circle':
      
      #check if valid motion
      delta =  float(self.linearPos) + trans_dist2img(velocity*time)
      if delta < self.linearExtent and delta > 0:
        self.linearPos += trans_dist2img(velocity*time)
      if delta >= self.linearExtent:
        if self.nextObj is not None:
          runoff = delta - self.linearExtent
          self.nextObj.getMotion(trans_img2dist(runoff), 1)
          self.linearPos = 0
          self.realPos = self.start
          print('traversing forward')
          return trans_img2dist(runoff)
          pass
        else:
          return 'a'

      if (self.linearPos + trans_dist2img(velocity*time) < 0):
        print('traversing backwards is not yet supported ¯\_(ツ)_/¯')
        pass
      
      v = velocity*self.approach_direction

      delta_t = np.arctan(trans_dist2img(v*time)/self.radius)
      self.theta = self.theta + delta_t
      
      (radX, radY) = self.radius_point
      (posX, posY) = self.trans_obj_2_img(radX + self.radius*np.cos(self.theta), radY + self.radius*np.sin(self.theta))

      self.realPos = (int(posX), int(posY))
      cv2.circle(track, self.realPos, 5, self.color, -1)
    else:
      pass

def trans_dist2img(x):
  scaler = 270
  return(int(x*scaler))
def trans_img2dist(x):
  scaler = (1/270)
  return(int(x*scaler))


In [968]:
# Contour Points
cps = [(-2.336, -1.133, 'line',   (0,255,0), (0,0),           0, 0),
       ( 1.754, -1.133, 'circle', (255,0,0), (1.754,0),       1, 1),
       ( 1.043,  0.882, 'line',   (0,0,255), (0,0),           0, 0),
       ( 0.182,  0.189, 'circle', (0,100,0), (-0.163,0.617),  0, -1),
       (-0.426,  0.134, 'line',   (100,0,0), (0,0),           0, 0),
       (-2.073,  1.031, 'circle', (0,0,100), (-2.336,0.548),  0, 1),
       (-2.885,  0.582, 'line',   (50,0,0),  (0,0),           0, 0),
       (-2.885, -0.582, 'circle', (0,50,0),  (-2.336,-0.582), 0, 1),
       (-2.336, -1.133, '',       (0,0,50),  (0,0),           0, 0)]
# Instantiate all the object, currently just lines and circles
objs = list()
i = 0
while i < len(cps)-1:
  start = (trans_dist2img(cps[i][0]), trans_dist2img(cps[i][1]))
  end = (trans_dist2img(cps[i+1][0]), trans_dist2img(cps[i+1][1]))
  
  objs.append(obj(cps[i][2], 
                  pic_center, 
                  start, 
                  end, 
                  cps[i][3], 
                  (trans_dist2img(cps[i][4][0]), trans_dist2img(cps[i][4][1])),
                  cps[i][5],
                  cps[i][6]))
  i +=1

# Assembling all the objects into a single polyLine (it's justa  linked list)
track_contour = polyLine()
track_contour.header = objs[0]
# Forward linking
i = 0
while i < len(objs)-1:
  objs[i].nextObj = objs[i+1]
  i+=1
# Backwards Linking
i = len(objs)-1
while i-1 > 0:
  objs[i].prevObj = objs[i-1]

  i-=1

In [ ]:
i = 0
print('start point', objs[i].start)

while i < len(objs)+1:
  objs[i].draw_start_end()
  runoff = objs[i].getMotion(1, 0.5)
  if runoff == 'a':
    break
  if runoff != None:
    i +=1
  
  print('real position', objs[i].realPos)
  print(objs[i].linearPos)
  print(objs[i].linearExtent)
  cv2_imshow(track)
  time.sleep(0.1)

